<a href="https://colab.research.google.com/github/talhaanwarch/Epilepsy_classification/blob/master/Epilepsy_v2_all_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://zenodo.org/record/1252141

In [ ]:
!pip install git+https://github.com/forrestbao/pyeeg.git
!pip install git+https://github.com/talhaanwarch/entropy.git


  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-cfjlrond
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-cfjlrond
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28122 sha256=bbf249a975a8d88bca316b78a18f39b4371c7d24cd39130b4d2f4a5da221db74
  Stored in directory: /tmp/pip-ephem-wheel-cache-ks2v4_ms/wheels/2d/3f/ad/106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg
  Cloning https://github.com/talhaanwarch/entropy.git to /tmp/pip-req-build-zbaqu956
  Running command git clone -q https://github.com/talhaanwarch/entropy.git /tmp/pip-req-build-zbaqu956
  Created wheel for entropy: filename=entropy-0.1.1-cp36-none-any.whl size=15448 sha256=9b260106acd0a47d9673124547354f484abccb7a0e340d08ab27fc5fff3c893f
  Stored in directory: /tmp/pip-ephem-wheel-cache-nf_qe4wj/wheels/79/43/63/3fa9c8c5f03fe3a78d71c1aeedca9577f2f32fb2156ff9ecf8
Successfully built entropy


In [ ]:
!wget https://zenodo.org/record/1252141/files/EEGs_Nigeria.zip

--2020-11-23 10:03:44--  https://zenodo.org/record/1252141/files/EEGs_Nigeria.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276402063 (264M) [application/octet-stream]
Saving to: ‘EEGs_Nigeria.zip’

EEGs_Nigeria.zip    100%[===================>] 263.60M  9.48MB/s    in 22s     

2020-11-23 10:04:07 (12.0 MB/s) - ‘EEGs_Nigeria.zip’ saved [276402063/276402063]



In [ ]:
from zipfile import ZipFile 
data = ZipFile('EEGs_Nigeria.zip')
data.extractall()

In [ ]:
import pandas as pd

In [ ]:
meta=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_nigeria.csv')

In [ ]:
meta.head()

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,11,1,26/9/2016 13:42,1,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-1.csv.gz


In [ ]:
EP_sub=meta['csv.file'][meta['Group']=='epilepsy']
CT_sub=meta['csv.file'][meta['Group']=='control']

In [ ]:
CT_sub

0        signal-6-1.csv.gz
1        signal-9-1.csv.gz
2       signal-10-1.csv.gz
3       signal-11-2.csv.gz
4       signal-11-1.csv.gz
              ...         
161    signal-573-1.csv.gz
162    signal-574-1.csv.gz
163    signal-575-1.csv.gz
164    signal-576-1.csv.gz
165    signal-577-1.csv.gz
Name: csv.file, Length: 97, dtype: object

In [ ]:
Epilepsy=[]
for i in EP_sub:
  Epilepsy.append(pd.read_csv('EEGs_Nigeria/{}'.format(i), compression='gzip')) 

In [ ]:
Control=[]
for i in CT_sub:
  try:
    Control.append(pd.read_csv('EEGs_Nigeria/{}'.format(i), compression='gzip')) 
  except:
      pass

In [ ]:
Epilepsy[0].head()

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,COUNTER,INTERPOLATED,GYROX,GYROY,RAW_CQ,CQ_CMS,CQ_F7,CQ_T7,CQ_O2,CQ_FC6,CQ_AF4,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4036,330.221053,4134,-213.473684,155.242105,3712,322.421053,3935,4169,67.547368,-74.505263,4170,3815,76.894737,46,0,1555,1771,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,2,4053,341.505263,4138,-211.936842,160.368421,3715,321.905263,3944,4173,80.368421,-65.778947,4173,3816,82.536842,47,0,1551,1771,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
2,3,4053,-344.757895,4137,-212.957895,156.778947,3713,322.421053,3939,4174,77.294737,-64.757895,4173,3816,82.021053,48,0,1555,1771,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
3,4,4044,343.557895,4136,-212.452632,147.042105,3713,323.452632,3930,4170,64.989474,-65.778947,4170,3813,75.863158,49,0,1555,1771,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
4,5,4049,-344.242105,4131,-213.989474,141.400000,3712,319.347368,3927,4170,62.421053,-62.189474,4166,3811,73.305263,50,0,1555,1771,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [ ]:
channels=Epilepsy[0].columns[1:15].tolist()


In [ ]:
from scipy import signal
nyq = 0.5 * 128
l=0.3
low = l / nyq
high = 30 / nyq
b, a = signal.butter(3, [low,high], 'band')

In [ ]:
EP_data=[]
for EP in Epilepsy:
  data=EP.iloc[:,1:15].values
  EP_data.append(signal.filtfilt(b, a, data.T).T)


In [ ]:
CT_data=[]
for CT in Control:
  data=CT.iloc[:,1:15].values
  CT_data.append(signal.filtfilt(b, a, data.T).T)


In [ ]:
def reshape(data):   
    shape=data.ravel().shape[0]
    s=shape%(14*1000)
    if s!=0:
        d=data.ravel()[s//2:-s//2].reshape(-1,1000,14)
        #return np.swapaxes(d,1,2)   
        return d
    else: 
        d1=data.reshape(-1,1000,14)
        #return np.swapaxes(d1,1,2)   
        return d1

In [ ]:

EP_reshaped=[]
for i in EP_data:
    EP_reshaped.append(reshape(i))
    
    
CT_reshaped=[]
for i in CT_data:
    CT_reshaped.append(reshape(i))

In [ ]:
EP_reshaped[0].shape

(34, 1000, 14)

In [ ]:
from scipy import stats
import pyeeg
from entropy import *
def mean(data):
    return np.mean(data,axis=0)
    
def std(data):
    return np.std(data,axis=0)

def ptp(data):
    return np.ptp(data,axis=0)

def var(data):
        return np.var(data,axis=0)

def minim(data):
      return np.min(data,axis=0)


def maxim(data):
      return np.max(data,axis=0)

def argminim(data):
      return np.argmin(data,axis=0)


def argmaxim(data):
      return np.argmax(data,axis=0)

def mean_square(data):
      return np.mean(data**2,axis=0)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=0))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=0)),axis=0)


def skewness(data):
    return stats.skew(data,axis=0)

def kurtosis(data):
    return stats.kurtosis(data,axis=0)

def zero_crossing(data):
    return np.argmax(np.diff(np.sign(data),axis=0),axis=0)

def app_epy(data):
    result=[]
    for i in data.T:
        result.append(app_entropy(i, order=2, metric='chebyshev'))
    return np.array(result)

def perm_epy(data):
    result=[]
    for i in data.T:
        result.append(perm_entropy(i, order=3, normalize=True))
    return np.array(result)

def svd_epy(data):
    result=[]
    for i in data.T:
        result.append(svd_entropy(i, order=3, delay=1, normalize=True))
    return np.array(result)

def spectral_epy(data):
    result=[]
    for i in data.T:
        result.append(spectral_entropy(i, 100, method='welch', normalize=True))
    return np.array(result)

def sample_epy(data):
    result=[]
    for i in data.T:
        result.append(sample_entropy(i, order=2, metric='chebyshev'))
    return np.array(result)


def katz(data):
    result=[]
    for i in data.T:
        result.append(katz_fd(i))
    return np.array(result)

def higuchi(data):
    result=[]
    for i in data.T:
        result.append(higuchi_fd(i))
    return np.array(result)


def petrosian(data):
    result=[]
    for i in data.T:
        result.append(petrosian_fd(i))
    return np.array(result)

def autogressiveModelParameters(data):
    feature = []
    for i in data.T:
        coeff, sig = alg.AR_est_YW(i, order=5)
        feature.append(np.mean(coeff))
    return np.array(feature)

def teager(x):
    for i in range(len(x)-1):
        return x[i]**2 - (x[i-1]*x[i+1])


def lziv_complex(data):
    result=[]
    for i in data.T:
        result.append(lziv_complexity(i))
    return np.array(result)

        
def hjorth_mobility(data):
    return np.divide(np.std(np.diff(data,axis=0),axis=0),np.std(data,axis=0))

def hjorth_complexity(data):
    return np.divide(hjorth_mobility(np.diff(data,axis=0)),  hjorth_mobility(data))      




def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data),zero_crossing(data),
                          app_epy(data),perm_epy(data),svd_epy(data),spectral_epy(data),sample_epy(data),
                          katz(data),higuchi(data),petrosian(data),teager(data),lziv_complex(data),
                          hjorth_mobility(data),hjorth_complexity(data)),axis=0)


In [ ]:
features1=[]
for f in CT_reshaped:
    feature=[]
    for i in f:
        feature.append(concatenate_features(i))
    features1.append(np.mean(np.array(feature),axis=0))
    

In [ ]:

features2=[]
for f in EP_reshaped:
    feature=[]
    for i in f:
        feature.append(concatenate_features(i))
    features2.append(np.mean(np.array(feature),axis=0))

In [ ]:
x1=np.array(features1)        
x2=np.array(features2)      

X=np.concatenate((x1,x2),axis=0)
y=np.concatenate(((np.zeros(x1.shape[0])),(np.ones(x2.shape[0]))))


In [ ]:
X.shape,y.shape

((221, 364), (221,))

#import libraries

In [ ]:
from scipy import stats
import pyeeg
from entropy import *
import pywt
#from nitime import algorithms as alg
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
#import nolds
np.warnings.filterwarnings('ignore')

# Classification

In [ ]:
#%% split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42,stratify=y)

In [ ]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:

clf=SVC(kernel='rbf')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred))
print('accuracy is ',accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      0.68      0.76        28
         1.0       0.80      0.92      0.86        39

    accuracy                           0.82        67
   macro avg       0.83      0.80      0.81        67
weighted avg       0.83      0.82      0.82        67

accuracy is  0.8208955223880597


# Feature

In [ ]:
feature_list=['mean','std','ptp','var','minim','maxim','argmin','argmax','mean_square','rms','abs_diffs_signal','skewness','kurtosis','zero_crossing',
'app_epy','perm_epy','svd_epy','spectral_epy','sample_epy','katz','higuchi','petrosian','teager','lziv_complex',
'hjorth_mobility','hjorth_complexity']

In [ ]:
feature=X.copy()
label=y.copy()
from sklearn.preprocessing import scale,maxabs_scale
feature=scale(feature)

In [ ]:
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y):
    Cs = [ 0.1, 1, 10,15,20,25,30,40,50,60,70,100,120]
    gammas = [0.0001,0.0005,0.001,0.005,0.008, 0.01, 0.1,0.3,0.5, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=10)
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_


svc_param_selection(feature,label)


{'C': 60, 'gamma': 0.0001}


0.7883399209486166

In [ ]:
clf = SVC(kernel="rbf", C=60,gamma=0.0001, probability=True)
acc = cross_val_score(clf,feature,label,cv=10)
precision = cross_val_score(clf,feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.7883399209486166 0.10529517411905429
average precision :  0.8137964731347085 0.10955729941045206
average recall :  0.8435897435897436 0.09743589743589744
average f1 score :  0.8227032515488286 0.08146927534831684


In [ ]:

def knn_param_selection(X, y, nfolds):
    n_neighbors  = [ 3,4,5,6,7,8,9]
    weights  = ['uniform','distance']
    metric=['minkowski','manhattan','euclidean']
    param_grid = {'n_neighbors': n_neighbors, 'weights' : weights,'metric':metric}
    grid_search =GridSearchCV( KNeighborsClassifier(), param_grid, cv=nfolds,n_jobs=-1)
    grid_search.fit(X, y)
    print(grid_search.best_score_)
    return grid_search.best_params_


knn_param_selection(feature,label,10)

0.742094861660079


{'metric': 'minkowski', 'n_neighbors': 8, 'weights': 'uniform'}

In [ ]:
clf = KNeighborsClassifier(metric= 'minkowski', n_neighbors=8, weights= 'uniform')
acc = cross_val_score(clf,feature,label,cv=10)
precision = cross_val_score(clf,feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.742094861660079 0.08624959683451179
average precision :  0.7659929286399875 0.0705633198268304
average recall :  0.8044871794871795 0.12721573356521265
average f1 score :  0.7796316416661244 0.07957815803546052


In [ ]:

from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.001,0.05,0.1, 1,2,3,4,5,8, 10,12,15]
    max_it= [200,300,400,500]
    param_grid = {'C': C,'max_iter':max_it}
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    print(grid_search.best_score_)
    return grid_search.best_params_


logistic_param_selection(feature,label,10)

0.783399209486166


{'C': 0.05, 'max_iter': 200}

In [ ]:

clf = LogisticRegression(max_iter=200,C=0.05)
acc = cross_val_score(clf,feature,label,cv=10)
precision = cross_val_score(clf,feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.783399209486166 0.11270784082110516
average precision :  0.7943339993339993 0.10142811986439923
average recall :  0.8512820512820513 0.10134694448190601
average f1 score :  0.8195238095238094 0.09307866069207099


In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
#skf = StratifiedKFold(n_splits=10, random_state=2020, shuffle=False)
def dtree_param_selection(X,y):
    #create a dictionary of all values we want to test
    param_grid = { 'criterion':['gini','entropy'],'max_features':["auto", "sqrt", "log2"],'max_depth': np.arange(2, 20),'random_state':[10,20,30,40]}
    # decision tree model
    dtree_model=DecisionTreeClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=10)
    #fit model to data
    dtree_gscv.fit(X, y)
    print(dtree_gscv.best_score_)
    #print(dtree_gscv.best_estimator_)
    return dtree_gscv.best_params_

dtree_param_selection(feature,label)

0.724308300395257


{'criterion': 'gini',
 'max_depth': 3,
 'max_features': 'auto',
 'random_state': 30}

In [ ]:
clf = DecisionTreeClassifier(criterion= 'gini', max_depth= 3, max_features= 'auto',random_state=30)
acc = cross_val_score(clf,feature,label,cv=10)
precision = cross_val_score(clf,feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.724308300395257 0.049139591933424355
average precision :  0.8107074040510573 0.11491204516101654
average recall :  0.7256410256410255 0.136187402306559
average f1 score :  0.7489728709293926 0.049178317632461416


In [ ]:
ch=14

In [ ]:
feature_selected=[]
feat_non_sorted=[]
score_non_sorted=[]
for i ,j in zip(range(0,feature.shape[1],ch),feature_list):
  acc=svc_param_selection(feature[:,i:i+ch],label)
  print(j," : ",acc)
  feat_non_sorted.append(j)
  score_non_sorted.append(acc)

{'C': 30, 'gamma': 0.008}
mean  :  0.6061264822134388
{'C': 20, 'gamma': 0.1}
std  :  0.725098814229249
{'C': 25, 'gamma': 0.1}
ptp  :  0.6711462450592884
{'C': 50, 'gamma': 0.1}
var  :  0.6482213438735178
{'C': 60, 'gamma': 0.01}
minim  :  0.6798418972332017
{'C': 120, 'gamma': 0.005}
maxim  :  0.6478260869565219
{'C': 1, 'gamma': 0.3}
argmin  :  0.5932806324110672
{'C': 0.1, 'gamma': 0.0001}
argmax  :  0.5792490118577074
{'C': 40, 'gamma': 0.1}
mean_square  :  0.6480237154150197
{'C': 20, 'gamma': 0.1}
rms  :  0.725098814229249
{'C': 20, 'gamma': 0.005}
abs_diffs_signal  :  0.7701581027667983
{'C': 1, 'gamma': 0.5}
skewness  :  0.6290513833992095
{'C': 50, 'gamma': 0.1}
kurtosis  :  0.5841897233201581
{'C': 1, 'gamma': 0.1}
zero_crossing  :  0.7015810276679841
{'C': 70, 'gamma': 0.005}
app_epy  :  0.7286561264822135
{'C': 100, 'gamma': 0.005}
perm_epy  :  0.7567193675889328
{'C': 50, 'gamma': 0.01}
svd_epy  :  0.8011857707509881
{'C': 20, 'gamma': 0.01}
spectral_epy  :  0.75158102766

In [ ]:
score,feat  = zip(*sorted(zip(score_non_sorted, feat_non_sorted),reverse=True))


In [ ]:
acc=0
deleted_item=[]
for i in range(1,20):
  feature_selected=list(feat[:i])
  # feature_selected=del_item(feature_selected,deleted_item)

  X_good=[]
  for key,val in zip(feature_list,range(0,feature.shape[1],ch)):
      for fe in feature_selected:
          if key==fe:     
              #print('key',key,'value',val,":",val+ch) 
              X_good.append(feature[:,val:val+ch])
  good_feature=np.concatenate((X_good),axis=1)
  #good_feature=np.concatenate((good_feature,non_eeg),1)


  acc_new=svc_param_selection(good_feature,label)
  print(i,' : ', acc_new)


{'C': 50, 'gamma': 0.01}
1  :  0.8011857707509881
{'C': 25, 'gamma': 0.01}
2  :  0.8466403162055336
{'C': 70, 'gamma': 0.001}
3  :  0.8511857707509881
{'C': 120, 'gamma': 0.0005}
4  :  0.8468379446640316
{'C': 20, 'gamma': 0.005}
5  :  0.8280632411067194
{'C': 50, 'gamma': 0.0005}
6  :  0.8239130434782608
{'C': 20, 'gamma': 0.001}
7  :  0.8191699604743082
{'C': 15, 'gamma': 0.001}
8  :  0.8243083003952568
{'C': 10, 'gamma': 0.001}
9  :  0.8152173913043477
{'C': 100, 'gamma': 0.001}
10  :  0.8191699604743083
{'C': 70, 'gamma': 0.001}
11  :  0.8375494071146246
{'C': 70, 'gamma': 0.001}
12  :  0.8195652173913043
{'C': 1, 'gamma': 0.01}
13  :  0.8057312252964428
{'C': 60, 'gamma': 0.001}
14  :  0.8197628458498023
{'C': 70, 'gamma': 0.0005}
15  :  0.8154150197628459
{'C': 50, 'gamma': 0.001}
16  :  0.8239130434782609
{'C': 40, 'gamma': 0.001}
17  :  0.8148221343873517
{'C': 50, 'gamma': 0.0005}
18  :  0.8154150197628459
{'C': 40, 'gamma': 0.001}
19  :  0.8193675889328065


In [ ]:
feature_selected=feat[:5]
X_good=[]
for key,val in zip(feature_list,range(0,feature.shape[1],ch)):
    for fe in feature_selected:
        if key==fe:     
            print('key',key,'value',val,":",val+ch) 
            X_good.append(feature[:,val:val+ch])

key abs_diffs_signal value 140 : 154
key svd_epy value 224 : 238
key petrosian value 294 : 308
key lziv_complex value 322 : 336
key hjorth_mobility value 336 : 350


In [ ]:
good_feature=np.concatenate((X_good),axis=1)
good_feature.shape

(221, 70)

In [ ]:
svc_param_selection(good_feature,label)


{'C': 20, 'gamma': 0.005}


0.8280632411067194

In [ ]:
clf = SVC(kernel="rbf", C=20,gamma=0.005, probability=True)
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.8280632411067194 0.08808998962207941
average precision :  0.8421299288946347 0.10524701417725987
average recall :  0.8826923076923077 0.06254846904296436
average f1 score :  0.8584563786482826 0.06912479377759939


In [ ]:
knn_param_selection(good_feature,label,10)

0.6796442687747036


{'metric': 'minkowski', 'n_neighbors': 6, 'weights': 'uniform'}

In [ ]:
clf = KNeighborsClassifier(metric= 'minkowski', n_neighbors=6, weights= 'uniform')
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.6796442687747036 0.10759576463862368
average precision :  0.7429453879453879 0.1201280708184419
average recall :  0.6955128205128205 0.11578456325761823
average f1 score :  0.7140914737436476 0.10233508332692709


In [ ]:
logistic_param_selection(good_feature,label,10)

0.7968379446640317


{'C': 1, 'max_iter': 200}

In [ ]:

clf = LogisticRegression(max_iter=200,C=1)
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.7968379446640317 0.08750634653357357
average precision :  0.8220908258408258 0.07852001608073533
average recall :  0.8358974358974359 0.1423942910452802
average f1 score :  0.8218586726172934 0.08405802590977393


In [ ]:
dtree_param_selection(good_feature,label)

0.715612648221344


{'criterion': 'entropy',
 'max_depth': 3,
 'max_features': 'log2',
 'random_state': 10}

In [ ]:
clf = DecisionTreeClassifier(criterion= 'entropy', max_depth= 3, max_features= 'log2',random_state=10)
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.715612648221344 0.10270388010083821
average precision :  0.7088569518716578 0.07775310556214832
average recall :  0.8583333333333334 0.13833238278231436
average f1 score :  0.7729668607254815 0.09631017138505021


#Feature Selection

#feature

In [ ]:
feature_list=['mean','std','ptp','var','minim','maxim','argmin','argmax','mean_square','rms','abs_diffs_signal','skewness','kurtosis','zero_crossing',
'app_epy','perm_epy','svd_epy','spectral_epy','sample_epy','katz','higuchi','petrosian','teager','lziv_complex',
'hjorth_mobility','hjorth_complexity']

In [ ]:
feature=X.copy()
label=y.copy()
from sklearn.preprocessing import scale,maxabs_scale
feature=scale(feature)

In [ ]:
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y):
    Cs = [ 0.1, 1, 10,15,20,25,30,40,50,60,70,100,120]
    gammas = [0.0001,0.0005,0.001,0.005,0.008, 0.01, 0.1,0.3,0.5, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=10)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_score_


svc_param_selection(feature,label)


0.792292490118577

In [ ]:
feature_selected=[]
feat_non_sorted=[]
score_non_sorted=[]
for i ,j in zip(range(0,feature.shape[1],ch),feature_list):
  acc=svc_param_selection(feature[:,i:i+ch],label)
  print(j," : ",acc)
  feat_non_sorted.append(j)
  score_non_sorted.append(acc)

mean  :  0.592687747035573
std  :  0.6391304347826087
ptp  :  0.6575098814229249
var  :  0.5978260869565217
minim  :  0.6575098814229249
maxim  :  0.6252964426877471
argmin  :  0.592885375494071
argmax  :  0.5881422924901185
mean_square  :  0.5978260869565217
rms  :  0.6393280632411067
abs_diffs_signal  :  0.7294466403162055
skewness  :  0.633201581027668
kurtosis  :  0.6015810276679843
zero_crossing  :  0.6788537549407114
app_epy  :  0.7247035573122529
perm_epy  :  0.6924901185770752
svd_epy  :  0.6841897233201581
spectral_epy  :  0.6972332015810276
sample_epy  :  0.715810276679842
katz  :  0.6887351778656127
higuchi  :  0.6614624505928854
petrosian  :  0.7061264822134388
teager  :  0.5881422924901185
lziv_complex  :  0.7294466403162055
hjorth_mobility  :  0.6978260869565217
hjorth_complexity  :  0.6749011857707511


In [ ]:
score,feat  = zip(*sorted(zip(score_non_sorted, feat_non_sorted),reverse=True))


In [ ]:
acc=0
deleted_item=[]
for i in range(1,20):
  feature_selected=list(feat[:i])
  # feature_selected=del_item(feature_selected,deleted_item)

  X_good=[]
  for key,val in zip(feature_list,range(0,feature.shape[1],ch)):
      for fe in feature_selected:
          if key==fe:     
              #print('key',key,'value',val,":",val+ch) 
              X_good.append(feature[:,val:val+ch])
  good_feature=np.concatenate((X_good),axis=1)
  #good_feature=np.concatenate((good_feature,non_eeg),1)


  acc_new=svc_param_selection(good_feature,label)
  print(i,' : ', acc_new)


1  :  0.7294466403162055
2  :  0.7786561264822134
3  :  0.7792490118577075
4  :  0.792490118577075
5  :  0.7794466403162055
6  :  0.7697628458498024
7  :  0.8148221343873517
8  :  0.8148221343873517
9  :  0.8150197628458498
10  :  0.8106719367588932
11  :  0.8104743083003954
12  :  0.8195652173913043
13  :  0.8110671936758894
14  :  0.8112648221343874
15  :  0.8110671936758894
16  :  0.8067193675889328
17  :  0.8067193675889328
18  :  0.7976284584980238
19  :  0.7976284584980238


#Feature Selection

In [ ]:
feature_list=['mean','std','ptp','var','minim','maxim','mean_square','rms','abs_diffs_signal','skewness','kurtosis','zero_crossing',
'app_epy','perm_epy','svd_epy','spectral_epy','sample_epy','katz','higuchi','petrosian','hjorth_mobility','hjorth_complexity']

In [ ]:

classifiers = [
    KNeighborsClassifier(),
    SVC(kernel="linear"),
    DecisionTreeClassifier(),
    LogisticRegression()]

In [ ]:
X_all=X.copy()
y_all=y.copy()
clf_feature_selected=[]
for clf in classifiers:
    feature_selected=[]
    for i in range(50):
        randomize = np.arange(len(X_all))
        np.random.shuffle(randomize)
        X_all = X_all[randomize]
        y_all = y_all[randomize]
        for i,j in zip(range(0,X_all.shape[1],ch),feature_list):
            X_new=X_all[:,i:i+ch]
            y_new=y_all.copy()
            X_train, X_test, y_train, y_test = train_test_split(X_new, y_all,test_size=0.2, random_state=42,stratify=y_all)
            sc=StandardScaler()
            X_train=sc.fit_transform(X_train)
            X_test=sc.transform(X_test)

            clf.fit(X_train,y_train)
            y_pred = clf.predict(X_test)  
            acc=metrics.accuracy_score(y_test, y_pred)
      #print("Accuracy of ",j,metrics.accuracy_score(y_test, y_pred))
            if acc>0.75:
                feature_selected.append(j)
    clf_feature_selected.append(feature_selected)        
 # print('-------------------------------------------------------------------')

In [ ]:

clf_list=['KNN','SVM','DT','LR']
from collections import Counter
clf_count=[]
for i,j in zip(range(4),clf_list):
    feature_selected=clf_feature_selected[i]
    count= {x:feature_selected.count(x) for x in feature_selected}
    c = Counter(count)
    #print(c.most_common())
    #clf_count.append(c.most_common(10))
    print('==================================================================================================')
    print(j,c.most_common(10))

KNN [('kurtosis', 8), ('hjorth_complexity', 6), ('katz', 6), ('higuchi', 5), ('svd_epy', 4), ('sample_epy', 3), ('spectral_epy', 3), ('perm_epy', 1), ('var', 1), ('abs_diffs_signal', 1)]
SVM [('hjorth_complexity', 28), ('spectral_epy', 26), ('minim', 12), ('higuchi', 12), ('svd_epy', 10), ('sample_epy', 9), ('ptp', 8), ('hjorth_mobility', 8), ('kurtosis', 8), ('petrosian', 5)]
DT [('minim', 7), ('svd_epy', 7), ('skewness', 4), ('higuchi', 4), ('hjorth_mobility', 2), ('ptp', 2), ('std', 2), ('perm_epy', 1), ('spectral_epy', 1), ('var', 1)]
LR [('hjorth_complexity', 23), ('spectral_epy', 18), ('svd_epy', 17), ('higuchi', 15), ('minim', 13), ('sample_epy', 12), ('petrosian', 11), ('std', 10), ('skewness', 10), ('kurtosis', 8)]


In [ ]:

from collections import Counter
clf_feature=[]
for i in range(4):
    feature_selected=clf_feature_selected[i]
    count= {x:feature_selected.count(x) for x in feature_selected}
    c = Counter(count)
    c={k: v / total for total in (sum(c.values(), 0.0),) for k, v in c.items()}

    for k,v in c.items():
        if v>.1:
            clf_feature.append(k)

In [ ]:

count= {x:clf_feature.count(x) for x in clf_feature}
c = Counter(count)
print(c.most_common())
good_feature=[]
for i in range(len(c.keys())):
      good_feature.append(c.most_common()[i][0])
print('===============================================================================================================')
print(good_feature)

[('hjorth_complexity', 3), ('higuchi', 2), ('spectral_epy', 2), ('svd_epy', 2), ('kurtosis', 1), ('katz', 1), ('minim', 1), ('skewness', 1)]
['hjorth_complexity', 'higuchi', 'spectral_epy', 'svd_epy', 'kurtosis', 'katz', 'minim', 'skewness']


In [ ]:

X_good=[]
for key,val in zip(feature_list,range(0,X.shape[1],ch)):
    for fe in good_feature:
        if key==fe:     
            print('key',key,'value',val,":",val+ch) 
            X_good.append(X[:,val:val+16])

key minim value 40 : 50
key skewness value 90 : 100
key kurtosis value 100 : 110
key svd_epy value 140 : 150
key spectral_epy value 150 : 160
key katz value 170 : 180
key higuchi value 180 : 190
key hjorth_complexity value 210 : 220


In [ ]:

X=np.concatenate((X_good),axis=1)


# Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

clf = SVC(kernel="rbf")
Cs = [ 1,2,3,5,7,10,15,30,50,70,100]
gammas = [0.001,0.01,0.02,0.03,0.04,0.05,0.1,0.3,0.5,0.8,1]
pipe = Pipeline(steps=[('scaler', sc), ('classifier', clf)])
estimator = GridSearchCV(pipe,
                         dict(classifier__C=Cs,classifier__gamma=gammas),cv=5,n_jobs=-1,iid=False) 

results = estimator.fit(X,y)
print(results.best_score_ )
results.best_params_

0.751010101010101


{'classifier__C': 50, 'classifier__gamma': 0.001}

### NuSVM

In [ ]:
from sklearn.svm import SVC, LinearSVC, NuSVC

clf = NuSVC(kernel="rbf")
nu = [0.1,0.3,0.5,0.7]
gammas = [0.001,0.01,0.02,0.03,0.04,0.05,0.1,0.3,0.5,0.8,1]
pipe = Pipeline(steps=[('scaler', sc), ('classifier', clf)])
estimator = GridSearchCV(pipe,
                         dict(classifier__nu=nu,classifier__gamma=gammas),cv=5,n_jobs=-1,iid=False) 

results = estimator.fit(X,y)
print(results.best_score_ )
results.best_params_

### LR

In [ ]:
clf = LogisticRegression()
C = [ 1,2,3,5,7,10,15,30,50,70,100]
penalty=["l1","l2"]
pipe = Pipeline(steps=[('scaler', sc), ('classifier', clf)])
estimator = GridSearchCV(pipe,
                         dict(classifier__C=C,classifier__penalty=penalty),cv=5,n_jobs=-1,iid=False) 

results = estimator.fit(X,y)
print(results.best_score_ )
results.best_params_

### KNN

In [ ]:

clf = KNeighborsClassifier()

n_neighbors  = [ 3,4,5,6,7,8,9,11,13]
weights  = ['uniform','distance']
metric=['minkowski','manhattan','euclidean']
pipe = Pipeline(steps=[('scaler', sc), ('classifier', clf)])
estimator = GridSearchCV(pipe,
                         dict(classifier__n_neighbors=n_neighbors,classifier__weights=weights,classifier__metric=metric),cv=5,n_jobs=-1,iid=False) 
results = estimator.fit(X,y)
print(results.best_score_ )
results.best_params_

# Cross Validation

In [ ]:
classifiers = [
    KNeighborsClassifier(n_neighbors=8,weights='distance',metric='manhattan'),
    SVC(kernel="rbf",C=70,gamma=0.001),
    NuSVC(nu=0.3,gamma=0.001),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression(C=3,penalty='l2')]

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('transformer', sc),('estimator', clf)])


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

accuracy=[]
accuracy_std=[]
f1_scores=[]
f1_scores_std=[]
for clfs in classifiers:
    pipeline.set_params(estimator = clfs)
    name = clfs.__class__.__name__
    
    print("="*30)
    print(name)       
    print('****Results****')
    scores = cross_val_score(pipeline, X, y, cv=5)
    #f1_score = cross_val_score(clf, X_rfe, y, cv=5,scoring='f1')
    print("Accuracy: {:.4%}".format(np.array(scores).mean()))
    accuracy.append(np.array(scores).mean())
    accuracy_std.append(np.array(scores).std())
    #f1_scores.append(np.array(f1_score).mean())
    #f1_scores_std.append(np.array(f1_score).std())


In [ ]:
import matplotlib.pyplot as plt
classifier=['KNN','SVC','nuSCV','DT','RF','Ada','GB','NB','LDA','QDA','LR']
y_pos = np.arange(len(classifier))
plt.bar(y_pos,np.array(accuracy))
plt.xticks(y_pos, classifier)
plt.ylabel('Accuracy')
plt.title('Classifiers')

# Principle Component Analysis

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import cross_val_score

#Fitting the PCA algorithm with our Data
pca = PCA().fit(X_train)
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('EEG')
plt.show()

In [ ]:
clf = SVC(kernel='rbf')
pca = PCA(n_components=5)

#pipeline = Pipeline([('selector',pca), ('estimator', clf)])
pipeline = Pipeline([('transformer', sc),('selector',pca), ('estimator', clf)])


In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

accuracy_pca=[]
std_pca=[]
f1_scores=[]
f1_scores_std=[]
for clfs in classifiers:
    pipeline.set_params(estimator = clfs)
    name = clfs.__class__.__name__
    
    print("="*30)
    print(name)       
    print('****Results****')
    scores = cross_val_score(pipeline, X, y, cv=5)
    #f1_score = cross_val_score(clf, X_rfe, y, cv=5,scoring='f1')
    print("Accuracy: {:.4%}".format(np.array(scores).mean()))
    accuracy_pca.append(np.array(scores).mean())
    std_pca.append(np.array(scores).std())
    #f1_scores.append(np.array(f1_score).mean())
    #f1_scores_std.append(np.array(f1_score).std())


In [ ]:
from sklearn.model_selection import GridSearchCV

clf = SVC(kernel="rbf")
n_components = [5,10,15,20]
Cs = [0.001,0.1, 1,2,3,5,7,10,15,30]
gammas = [0.001,0.01,0.02,0.03,0.04,0.05,0.1,0.3,0.5,0.8,1]
pipe = Pipeline(steps=[('scalar',sc),('selector', pca), ('classifier', clf)])
estimator = GridSearchCV(pipe,
                         dict(selector__n_components=n_components,
                              classifier__C=Cs,classifier__gamma=gammas),cv=5,n_jobs=-1,iid=False) 

results = estimator.fit(X,y)
print(results.best_score_ )
results.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = KNeighborsClassifier()
n_components = [5,10,15,20]
n_neighbors  = [ 3,4,5,6,7,8,9]
weights  = ['uniform','distance']
metric=['minkowski','manhattan','euclidean']
pipe = Pipeline(steps=[('scalar',sc),('selector', pca), ('classifier', clf)])
estimator = GridSearchCV(pipe,
                         dict(selector__n_components=n_components,
                              classifier__n_neighbors=n_neighbors,classifier__weights=weights,classifier__metric=metric),cv=5,n_jobs=-1,iid=False) 

results = estimator.fit(X,y)
print(results.best_score_ )
results.best_params_

# RFE

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:

clf = SVC(kernel='rbf')
rfe=RFE(SVC(kernel='linear'),n_features_to_select=5,step=1,verbose=0)   

#pipeline = Pipeline([('selector',rfe), ('estimator', clf)])
pipeline = Pipeline([('transformer', sc),('selector',rfe), ('estimator', clf)])


In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

accuracy_rfe=[]
std_rfe=[]
f1_scores=[]
f1_scores_std=[]
for clfs in classifiers:
    pipeline.set_params(estimator = clfs)
    name = clfs.__class__.__name__
    
    print("="*30)
    print(name)       
    print('****Results****')
    scores = cross_val_score(pipeline, X, y, cv=5)
    #f1_score = cross_val_score(clf, X_rfe, y, cv=5,scoring='f1')
    print("Accuracy: {:.4%}".format(np.array(scores).mean()))
    accuracy_rfe.append(np.array(scores).mean())
    std_rfe.append(np.array(scores).std())
    #f1_scores.append(np.array(f1_score).mean())
    #f1_scores_std.append(np.array(f1_score).std())


# SKbest

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


In [ ]:

clf = SVC(kernel='rbf')
skbest=SelectKBest(f_classif, k=10)
#pipeline = Pipeline([('selector',skbest), ('estimator', clf)])
pipeline = Pipeline([('transformer', sc),('selector',rfe), ('estimator', clf)])


In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

accuracy_kbest=[]
std_kbest=[]
f1_scores=[]
f1_scores_std=[]
for clfs in classifiers:
    pipeline.set_params(estimator = clfs)
    name = clfs.__class__.__name__
    
    print("="*30)
    print(name)       
    print('****Results****')
    scores = cross_val_score(pipeline, X, y, cv=5)
    #f1_score = cross_val_score(clf, X_rfe, y, cv=5,scoring='f1')
    print("Accuracy: {:.4%}".format(np.array(scores).mean()))
    accuracy_kbest.append(np.array(scores).mean())
    std_kbest.append(np.array(scores).std())
    #f1_scores.append(np.array(f1_score).mean())
    #f1_scores_std.append(np.array(f1_score).std())


# VarianceThreshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold
clf = SVC(kernel='rbf')
selector = VarianceThreshold(0.05)
#pipeline = Pipeline([('selector',algo), ('estimator', clf)])
pipeline = Pipeline([('transformer', sc),('selector',selector), ('estimator', clf)])


In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

accuracy_var=[]
std_var=[]
f1_scores=[]
f1_scores_std=[]
for clfs in classifiers:
    pipeline.set_params(estimator = clfs)
    name = clfs.__class__.__name__
    
    print("="*30)
    print(name)       
    print('****Results****')
    scores = cross_val_score(pipeline, X, y, cv=5)
    #f1_score = cross_val_score(clf, X_rfe, y, cv=5,scoring='f1')
    print("Accuracy: {:.4%}".format(np.array(scores).mean()))
    accuracy_var.append(np.array(scores).mean())
    std_var.append(np.array(scores).std())
    #f1_scores.append(np.array(f1_score).mean())
    #f1_scores_std.append(np.array(f1_score).std())


# Result

In [ ]:
len(accuracy),len(accuracy_pca),len(accuracy_rfe),len(accuracy_kbest),len(accuracy_var)

In [ ]:
result=pd.DataFrame([accuracy,accuracy_pca,accuracy_rfe,accuracy_kbest,accuracy_var],
                    index=['Accuracy','PCA',"RFE",'kbest','VAR'],
                    columns=['KNN','SVC','nuSCV','DT','RF','Ada','GB','NB','LDA','QDA','LR'])

In [ ]:
result=result.T

In [ ]:

def _color_red_or_green(val):
    color = 'red' if val < 0.7 else 'green'
    return 'color: %s' % color
result.style.applymap(_color_red_or_green)

In [ ]:
#Higest accuracy 90% check PCA section or following code